# Useful notebook to run to test that all figure creation works


This is not testing for software bugs, but for problems with the data that may result in 
series  of NaNs or infs, which may trigger an exception with matplotlib (typically
when there is no number in a data set to plot, and we set the axis labels to be NaN 
or inf).



What is better than running 'generate_countries.ipynb'?

- we don't produce html here, and the process is sligthly faster
- running 'generate_countries.ipynb' is a little bit indirect:
  - we create a notebook template, then execute the template to create html
  - in at least one instance, there was an error in the figure creation
    (reported by matplotlib) that did not result in an exception, i.e.
    that was only discovered by looking at one of the ~600 html files manually

In [ ]:
from multiprocessing import Pool, cpu_count
import time
import matplotlib
matplotlib.use("Agg")

#  Set to false if you do not want multiprocessing enabled
cores = '1'  # not implemented yet
if cores == 'auto':
    cores = max(1, cpu_count()-1)

if cores:
    print(f'Using {cores} processes')

import test_figure_creation

test_figure_creation.test_world_overview()

In [ ]:
test_figure_creation.test_germany_overview()